# Installing All Libraries Used in Code

In [1]:
# !pip install pandas
# !pip install opencv-python
# !pip install numpy
# !pip install tensorflow
# !pip install keras
# !pip install sklearn
# !pip install imblearn
# !pip install matplotlib
# !pip install seaborn
# !pip install scipy
# !pip install librosa
# !pip install IPython

# Importing important libraries

In [2]:
import pandas as pd
import cv2
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import librosa
import librosa.display
import IPython.display as ipd

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


# Reading CSV data file 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
audios = os.listdir('/content/drive/MyDrive/CureMD/Audios')

In [5]:
df = pd.read_csv("/content/drive/MyDrive/CureMD/overview-of-recordings.csv")
df.head()

,audio_clipping,audio_clipping:confidence,background_noise_audible,background_noise_audible:confidence,overall_quality_of_the_audio,quiet_speaker,quiet_speaker:confidence,speaker_id,file_download,file_name,phrase,prompt,writer_id
0,no_clipping,1.0000,light_noise,1.0000,3.33,audible_speaker,1.0,43453425,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43453425_58166571.wav,When I remember her I feel down,Emotional pain,21665495
1,light_clipping,0.6803,no_noise,0.6803,3.33,audible_speaker,1.0,43719934,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43719934_43347848.wav,When I carry heavy things I feel like breaking...,Hair falling out,44088126
2,no_clipping,1.0000,no_noise,0.6655,3.33,audible_speaker,1.0,43719934,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43719934_53187202.wav,there is too much pain when i move my arm,Heart hurts,44292353
3,no_clipping,1.0000,light_noise,1.0000,3.33,audible_speaker,1.0,31349958,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_31349958_55816195.wav,My son had his lip pierced and it is swollen a...,Infected wound,43755034
4,no_clipping,1.0000,no_noise,1.0000,4.67,audible_speaker,1.0,43719934,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43719934_82524191.wav,My muscles in my lower back are aching,Infected wound,21665495


# Converting Audio files to Spectograms with static, delta, and delta-delta features

In [97]:
y = []
images = []
for j in range(len(audios)):
  print(j)
  x, sr = librosa.load(f'{os.getcwd()}/drive/MyDrive/CureMD/Audios/{audios[j]}', sr=44100)
  static = librosa.feature.mfcc(y=x, sr=44100)
  delta = librosa.feature.delta(static)
  delta2 = librosa.feature.delta(static, order=2)

  # X = librosa.stft(x)
  # Xdb = librosa.amplitude_to_db(abs(X))
  images.append(cv2.resize(np.dstack((static,delta,delta2)), (20,300), interpolation = cv2.INTER_AREA))
  y.append(df[df['file_name'] == audios[j]].prompt)
  # plt.colorbar()
  # #         print(i,j)
  # plt.savefig(f'{os.getcwd()}\\images_original\\{categories[i]}\\{image_list[j]}'[:-3]+'png')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# Normalizing Data

In [106]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

images = np.array(images) / 255
images

array([[[[-1.19197663e+00,  5.74380685e-04,  1.41127933e-04],
         [-1.19197663e+00,  5.74380685e-04,  1.41127933e-04],
         [-1.19197663e+00,  5.74380685e-04,  1.41127933e-04],
         ...,
         [-1.17510208e+00,  2.23901342e-03, -2.76598790e-03],
         [-1.17510208e+00,  2.23901342e-03, -2.76598790e-03],
         [-1.17510208e+00,  2.23901342e-03, -2.76598790e-03]],

        [[ 6.07451256e-01, -5.50130185e-04,  4.38350042e-03],
         [ 6.07451256e-01, -5.50130185e-04,  4.38350042e-03],
         [ 6.07451256e-01, -5.50130185e-04,  4.38350042e-03],
         ...,
         [ 6.20473944e-01,  2.91705272e-03, -4.35360366e-03],
         [ 6.20473944e-01,  2.91705272e-03, -4.35360366e-03],
         [ 6.20473944e-01,  2.91705272e-03, -4.35360366e-03]],

        [[ 7.00293447e-02, -1.16725506e-03,  8.18357094e-03],
         [ 7.00293447e-02, -1.16725506e-03,  8.18357094e-03],
         [ 7.00293447e-02, -1.16725506e-03,  8.18357094e-03],
         ...,
         [ 7.85987779e-0

# Separating Features and Target Column

In [108]:
y2 = []
for i in range(len(y)):
  y2.append(y[i].to_numpy()[0])

In [109]:
y = pd.get_dummies(y2)

In [110]:
y

,Acne,Cough,Feeling dizzy,Joint pain,Knee pain,Shoulder pain
0,0,1,0,0,0,0
1,1,0,0,0,0,0
2,0,0,0,0,1,0
3,1,0,0,0,0,0
4,0,0,1,0,0,0
...,...,...,...,...,...,...
1842,0,0,0,0,0,1
1843,0,0,0,0,0,1
1844,0,0,0,0,0,1
1845,1,0,0,0,0,0


# Spliting Train Set and Test Set 

In [115]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(images), np.array(y), test_size=0.20,random_state = 42)


In [116]:
pd.DataFrame(y_train).value_counts()

0  1  2  3  4  5
0  0  0  0  0  1    271
1  0  0  0  0  0    264
0  0  0  1  0  0    250
         0  1  0    244
   1  0  0  0  0    227
   0  1  0  0  0    221
dtype: int64

In [117]:
pd.DataFrame(y_test).value_counts()

0  1  2  3  4  5
0  0  0  1  0  0    68
   1  0  0  0  0    66
1  0  0  0  0  0    64
0  0  1  0  0  0    62
      0  0  1  0    61
            0  1    49
dtype: int64

# Building Model

In [126]:
from tensorflow.keras import regularizers

model1 = Sequential()

model1.add(ZeroPadding2D((1,1),input_shape=images[0].shape))
model1.add(Convolution2D(32, 3, 3, activation='relu',))
model1.add(ZeroPadding2D((1,1)))
model1.add(Convolution2D(64, 3, 3, activation='relu',))
model1.add(MaxPooling2D((2,2), padding='same'))
model1.add(Flatten())
model1.add(Dense(2048, activation='relu',))
model1.add(Dropout(0.1))
model1.add(Dense(512, activation='relu',))
model1.add(Dropout(0.1))
model1.add(Dense(128, activation='relu',))
model1.add(Dropout(0.1))
model1.add(Dense(32, activation='relu',))
model1.add(Dropout(0.1))
model1.add(Dense(len(y.columns), activation='softmax'))


In [119]:
model1.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d_13 (ZeroPadd  (None, 22, 302, 3)       0         
 ing2D)                                                          
                                                                 
 conv2d_13 (Conv2D)          (None, 7, 100, 32)        896       
                                                                 
 zero_padding2d_14 (ZeroPadd  (None, 9, 102, 32)       0         
 ing2D)                                                          
                                                                 
 conv2d_14 (Conv2D)          (None, 3, 34, 64)         18496     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 2, 17, 64)        0         
 2D)                                                             
                                                      

In [127]:
opt = tf.keras.optimizers.Adam()
model1.compile(loss='categorical_crossentropy', optimizer=opt,metrics='accuracy')
model1.fit(X_train, y_train,
          epochs=100,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/100
47/47 [==============================] - 6s 105ms/step - loss: 1.7974 - accuracy: 0.1625 - val_loss: 1.7924 - val_accuracy: 0.1730
Epoch 2/100
47/47 [==============================] - 5s 97ms/step - loss: 1.7912 - accuracy: 0.1787 - val_loss: 1.7944 - val_accuracy: 0.1730
Epoch 3/100
47/47 [==============================] - 4s 95ms/step - loss: 1.7904 - accuracy: 0.1787 - val_loss: 1.7938 - val_accuracy: 0.1730
Epoch 4/100
47/47 [==============================] - 4s 90ms/step - loss: 1.7870 - accuracy: 0.1801 - val_loss: 1.7923 - val_accuracy: 0.1811
Epoch 5/100
47/47 [==============================] - 4s 93ms/step - loss: 1.7875 - accuracy: 0.1598 - val_loss: 1.7937 - val_accuracy: 0.1324
Epoch 6/100
47/47 [==============================] - 5s 100ms/step - loss: 1.7838 - accuracy: 0.1903 - val_loss: 1.7954 - val_accuracy: 0.1595
Epoch 7/100
47/47 [==============================] - 4s 93ms/step - loss: 1.7790 - accuracy: 0.1991 - val_loss: 1.7902 - val_accuracy: 0.1784
Epoc